In [ ]:
# Kết nối COLAB với GDrive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Đọc dữ liệu
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/Bank_Detection/creditcard.csv")
df.head()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Đếm số dòng dữ liệu
df.count()

In [ ]:
# Số dòng dữ liệu mỗi class
df['Class'].value_counts()

**Train model auto Encoder**

In [ ]:
# Drop cột time
df_no_time = df.drop(columns=['Time'])


In [ ]:
df_no_time.head()

In [ ]:
# Phân chia dữ liệu train test
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(df_no_time , test_size = 0.2, random_state=42)

# Diều chế dữ liệu train
X_train = X_train[X_train['Class']==0]
X_train = X_train.drop(columns=['Class'])

# Điều chế X_test
y_test = X_test['Class']
X_test = X_test.drop(columns=['Class'])

X_train = X_train.values
X_test = X_test.values


In [ ]:
from sklearn.preprocessing import StandardScaler

df_no_time ['Amount'] = StandardScaler().fit_transform(df_no_time['Amount'].values.reshape(-1,1))


In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
from keras.layers import Input, Dense
from keras.models import load_model, Model
from keras.callbacks import ModelCheckpoint

In [ ]:


# Xây dựng model
input_dimension = X_train.shape[1]
hidden_size = 128

input_layer = Input(shape=(input_dimension,))
# Encoder
encoder = Dense(hidden_size, activation = "relu") (input_layer)
encoder = Dense(hidden_size //2, activation="relu") (encoder)
encoder = Dense(hidden_size //4, activation="relu") (encoder)

# Decoder
decoder = Dense(hidden_size //4, activation="relu") (encoder)
decoder = Dense(hidden_size //2, activation="relu") (decoder)
decoder = Dense(hidden_size, activation = "relu") (decoder)

# Output
output_layer = Dense(input_dimension, activation="relu") (decoder)

auto_encoder_model= Model(inputs = input_layer, outputs = output_layer)
auto_encoder_model.compile (optimizer ="adam", loss="mse", metrics=['accuracy'])
auto_encoder_model.summary()

In [ ]:
# Train model
n_epochs = 30
n_batch_size = 32

save_best = ModelCheckpoint(filepath = "/content/drive/MyDrive/Bank_Detection/best.h5",
                            monitor='val_loss', verbose=1, save_best_only=True)

history = auto_encoder_model.fit(X_train, X_train, batch_size=n_batch_size, epochs=n_epochs,verbose=1,
                                 shuffle=True, callbacks=[save_best],
                                 validation_data=(X_test, X_test))



In [ ]:
# In history

import matplotlib.pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend()

plt.xlabel("epoch")
plt.ylabel("loss")
plt.show()

In [ ]:
# Kiểm tra việc tái tạo trong tập test
import numpy as np

# Dự đoán
predicts = auto_encoder_model.predict(X_test)
mse = np.mean(np.power(X_test - predicts,2), axis=1)
err_df = pd.DataFrame({'error': mse, 'truth': y_test})


In [ ]:
# In ra histogram của error khi tái tạo các điểm Normal trong tập test
normal_error_df = err_df[(err_df['truth']==0) & (err_df['error'] < 10)]

_ = plt.hist(normal_error_df.error.values, bins= 50)

In [ ]:
# In ra histogram của error khi tái tạo các điểm Fraud trong tập test
fraud_error_df = err_df[(err_df['truth']==1)]
_ = plt.hist(fraud_error_df.error.values, bins= 100)

In [ ]:
# Vẽ ra confusion matrix
from sklearn.metrics import confusion_matrix
import seaborn as sns

threshold = 2.5

y_pred = [1 if e > threshold else 0 for e in err_df.error.values]
conf_matrix = confusion_matrix(err_df.truth, y_pred)

# Vẽ
sns.heatmap(conf_matrix, xticklabels=["Normal", "Fraud"], yticklabels=['Normal','Fraud'], annot=True, fmt="d")
plt.title("Confusion Matrix")
plt.xlabel("Predited Values")
plt.ylabel("Truth")
plt.show